In [37]:
import pandas as pd
import numpy as np
import re
from urllib.parse import urlparse
import string

import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('words')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize, TweetTokenizer
from nltk.corpus import words, stopwords
from nltk.metrics.distance import jaccard_distance, edit_distance
from nltk.util import ngrams
from nltk.stem import WordNetLemmatizer

[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>
[nltk_data] Error loading omw-1.4: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>
[nltk_data] Error loading words: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>


In [21]:
train_df = pd.read_csv('/Users/ananyathomas/Documents/GitHub/case_shared_task_3/codefiles/subtask_2_data/train_subtask2.csv')
train_df.head()

,corpus,doc_id,sent_id,eg_id,index,text,text_w_pairs,seq_label,pair_label,context,num_sents
0,cnc,train_05_299,259,0,cnc_train_05_299_259_0,"ADILABAD : TRS condemns arrests March 11 , 201...","ADILABAD : TRS condemns arrests March 11 , 201...",1,1,NaN,1
1,cnc,train_09_B_24,2505,0,cnc_train_09_B_24_2505_0,"However , local villagers resisted the searche...","However , <ARG1>local villagers resisted the s...",1,1,NaN,1
2,cnc,train_02_109,291,0,cnc_train_02_109_291_0,"In 2009 , riots broke out in the capital , Uru...","In 2009 , <ARG0>riots broke out in the capital...",1,1,NaN,1
3,cnc,train_05_257,2109,0,cnc_train_05_257_2109_0,"The 60 - page charge sheet , filed by National...","The 60 - page charge sheet , filed by National...",1,1,NaN,1
4,cnc,train_05_223,1332,0,cnc_train_05_223_1332_0,TREATMENT Residents of Ravensmead on the Cape ...,TREATMENT <ARG1>Residents of Ravensmead on the...,1,1,NaN,1


In [22]:
train_grouped_df = pd.read_csv('/Users/ananyathomas/Documents/GitHub/case_shared_task_3/codefiles/subtask_2_data/train_subtask2_grouped.csv')
train_grouped_df.head()

,corpus,doc_id,sent_id,eg_id,index,text,causal_text_w_pairs,num_rs
0,cnc,train_01_0,892,0,cnc_train_01_0_892_0,The State alleged they hacked Sabata Petros Ch...,['The State alleged <ARG1>they hacked Sabata P...,1
1,cnc,train_01_1,2714,0,cnc_train_01_1_2714_0,Chale was allegedly chased by a group of about...,[],0
2,cnc,train_01_10,2619,0,cnc_train_01_10_2619_0,The farmworkers ' strike resumed on Tuesday wh...,"[""<ARG1>The farmworkers ' strike resumed on Tu...",1
3,cnc,train_01_100,2680,0,cnc_train_01_100_2680_0,Demonstrators have filed for a permit to hold ...,['<ARG1>Demonstrators have filed for a permit<...,2
4,cnc,train_01_101,3090,0,cnc_train_01_101_3090_0,"Footage of the attack , which included a pregn...","['<ARG0>Footage of the attack , which included...",2


In [23]:
print(train_df.isna().sum())
print(train_df.shape)
print(train_df.dtypes)

corpus             0
doc_id             0
sent_id            0
eg_id              0
index              0
text               0
text_w_pairs       0
seq_label          0
pair_label         0
context         2257
num_sents          0
dtype: int64
(2257, 11)
corpus           object
doc_id           object
sent_id           int64
eg_id             int64
index            object
text             object
text_w_pairs     object
seq_label         int64
pair_label        int64
context         float64
num_sents         int64
dtype: object


In [24]:
pip install contractions

Note: you may need to restart the kernel to use updated packages.


In [25]:
pip install num2words

Note: you may need to restart the kernel to use updated packages.


In [26]:
import contractions
import string
import num2words

In [27]:
abbreviations = {
    'n\'t':'not',
    'won\'t':'will not',
    'can\'t':'can not',
    'i\'m':'i am',
    'you\'re':'you are',
    'he\'s':'he is',
    'she\'s':'she is',
    'it\'s':'it is',
    'we\'re':'we are',
    'they\'re':'they are',
    'i\'ve':'i have',
    'you\'ve':'you have',
    'we\'ve':'we have',
    'NaMo':'Narendra Modi',
    'PMO':'Prime Minister Office',
    'PM':'Prime Minister',
    'BJP':'Bharatiya Janata Party',
    'INC':'Indian National Congress',
    'AAP':'Aam Aadmi Party',
    'J&K':'Jammu and Kashmir',
    'NHAI':'National Highways Authority of India',
    'CPI':'Communist Party of India',
    'CPI(M)':'Communist Party of India (Marxist)',
    'CPI-M':'Communist Party of India (Marxist)',
    'HQs':'Headquarters',
    'NGO':'Non Governmental Organization',
    'NGOs':'Non Governmental Organizations',
    'TRS':'Telangana Rashtra Samithi',
    'IST':'Indian Standard Time',
    'DA': 'Dearness Allowance',
    'MP':'Member of Parliament',
    'DGP':"Director General of Police",
    'WCED': "World Commission on Environment and Development",
    'JCB': "Joseph Cyril Bamford",
    'CPI': "Consumer Price Index",
    'ADM': "Additional District Magistrate",
    'PIL': "Public Interest Litigation",
    'PTI': "Press Trust of India",
    'TNCC': "Tamil Nadu Congress Committee",
    'JAC': "Joint Action Committee",
    'CPM': "Cost per thousand",
    'KCR': "Kalvakuntla Chandrashekar Rao",
    'NMMU': "Nelson Mandela Metropolitan University", 

}

In [38]:
def removing_links(text):
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(text)
    x = [word for word in tokens if not urlparse(word).scheme]
    text = ' '.join(x)
    return text

def contractions_handling(text):
    text = contractions.fix(text)
    return text

def replacing_abbr(text, dictry):
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(text)
    x = [dictry[word] if word in dictry.keys() else word for word in tokens]
    text = ' '.join(x)
    return text

def removing_punctuations(text):
    text = text.translate(str.maketrans(' ', ' ', string.punctuation))
    text = text.lower()
    return text

def removing_stopwords(text):
    #tokenizer = TweetTokenizer()
    stop_words = set(stopwords.words('english')) 
    tokens = word_tokenize(text)
    #tokens = tokenizer.tokenize(text)
    x = [word for word in tokens if not word.lower() in stop_words]
    x = []
    for w in tokens:
        if w not in stop_words:
            x.append(w)
    return text

def convert_num_words(text):
    re_results = re.findall('(\d+(st|nd|rd|th))', text)
    for entire_result, suffix in re_results:
        num = int(entire_result[:-len(suffix)])
        entire_result = " " + entire_result
        num_word = num2words.num2words(num, ordinal=True)
        text = text.replace(entire_result, " " + num_word)
    
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(text)
    tokens = [num2words.num2words(word) if word.isdigit() else word for word in tokens]
    text = ' '.join(tokens)
    
    return text

def removing_char_less_3(text):
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(text)
    tokens = [word for word in tokens if len(word) >= 3]
    text = ' '.join(tokens)
    return text

In [39]:
train_df['text'] = train_df['text'].apply(lambda x: removing_links(x))
train_df['text'] = train_df['text'].apply(lambda x: contractions_handling(x))
train_df['text'] = train_df['text'].apply(lambda x: replacing_abbr(x, abbreviations))
train_df['text'] = train_df['text'].apply(lambda x: removing_punctuations(x))
#train_df['text'] = train_df['text'].apply(lambda x: removing_stopwords(x))
train_df['text'] = train_df['text'].apply(lambda x: convert_num_words(x))
train_df['text'] = train_df['text'].apply(lambda x: removing_char_less_3(x))

train_df.head()

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/Users/ananyathomas/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.10/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.10/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.10/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************
